# IMDB Scraping tool
This is a scraping script that gets data for every episode of a tv show from the website IMDB.com.
It consists of two scraping functions that get relevant data and produces a dataframe and a excel file ready to be analyzed.


This script gets ids of tv shows from imdb top 50 chart and converts them to list that is readable by scraping functions

In [2]:
#necessary imports
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import OrderedDict
#get tv rating
lista=[]
def get_ids():
    page = requests.get(
        "https://www.imdb.com/chart/toptv/?sort=nv,desc&mode=simple&page=1")
    soup = BeautifulSoup(page.content, 'html5lib')
    
    #parse name of the series
    
    for h1 in soup.find_all('td', class_="titleColumn"):
        lista.append(h1.find("a")['href'])
        
    #print(soup)
    return lista
    
    
get_ids()

final=[]
for i in lista:
    final.append(i.split("/title/")[1].split("/")[0])
    
final1=final[25:50]
final1

['tt0141842',
 'tt0306414',
 'tt3032476',
 'tt2861424',
 'tt1124373',
 'tt0285403',
 'tt0303461',
 'tt0098904',
 'tt1119644',
 'tt1442449',
 'tt2243973',
 'tt0149460',
 'tt0106179',
 'tt0436992',
 'tt0417299',
 'tt0804503',
 'tt1439629',
 'tt0877057',
 'tt0472954',
 'tt1266020',
 'tt0979432',
 'tt1586680',
 'tt0387199',
 'tt1839578',
 'tt0098936']

First scraping function that gets following data from the website:

    Name of episode
    Length of episode
    Release year
    Number of ratings on the website
    Genre
    Series name (TV show name)

Also there is a list of ids of my favourite tv shows provided (idnlist), which is used to test the function.

In [3]:
#get name+rating from advanced view
# http://www.imdb.com/search/title/?series=tt5296406&count=250&view=advanced&sort=release_date,asc&ref_=tt_eps_rhs_sm
# compare name and append to the dataframe 

idnlist=['tt0903747',
'tt1632701',
'tt4158110',
'tt2575988',
'tt3322314',
'tt3322312',
'tt2357547',
'tt3322310',
'tt5675620',
'tt4230076',
'tt3032476',
'tt2442560',
'tt1837492',
'tt5420376',
'tt4270492',
'tt1475582',
'tt4800878',
'tt2356777',
'tt2707408',
'tt2802850',
'tt0475784',
'tt4574334',
'tt5753856',
'tt2085059',
'tt2261227',
'tt5071412',
'tt5290382',
'tt5028002',
'tt4118466',
'tt0290978',
'tt4422836',
'tt5296406']

def parse_and_get_df2(idn):
    page = requests.get(
        "http://www.imdb.com/search/title/?series={}&count=250&view=advanced&sort=release_date,asc&ref_=tt_eps_rhs_sm".format(idn))
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #parse name of the series
    series=soup.select('h3 > a')
    nams=series[0].get_text().strip()
    #parse name of each episode
    listname=[]
    for name in soup.select('h3 > a'):
        listname.append(name.get_text())
    listname2=listname[1::2]
    #parse length of each episode
    length=[]
    for name in soup.select('p > span.runtime'):
        length.append(name.get_text())
    #parse year
    year=[]
    table=soup.findAll('span', {"class": "lister-item-year text-muted unbold" })
    yearlist=table[1::2]
    for i in yearlist:
        year.append(i.get_text())
    year=[i.replace('(', '').replace(')', '') for i in year] #convert strings to int
    #print(year)
    #parse type of the tv show
    
    x=soup.find('span', {'class':'genre'})
    x=x.get_text().strip()
    #print(x)
    
   #get number of vote
    votes=[]
    z=soup.findAll('span', {'name':'nv'})
    for i in z:
        votes.append(i.get_text())
    #print(votes)
    
    
    finname=listname2[0:len(length)] #this is the shortest list one that is available
    finyear=year[0:len(length)]
    finvotes=votes[0:len(length)]
    #print(finname)
    #print(length)

    df= pd.DataFrame( OrderedDict( (('name', pd.Series(finname)), 
                                    ('len', pd.Series(length)),
                                    ('year', pd.Series(finyear)),
                                    ('votes', pd.Series(finvotes))
                                   ) ) )
    df['genre']=x
    df['series']=nams
    #df['cert']=cert
    
    return df

In [16]:
%%time
#test function and check speed
df=parse_and_get_df2('tt0386676')
print(df)

                               name     len  year  votes   genre series
0                             Pilot  23 min  2005  2,936  Comedy  Biuro
1                     Diversity Day  23 min  2005  2,811  Comedy  Biuro
2                       Health Care  22 min  2005  2,339  Comedy  Biuro
3                      The Alliance  23 min  2005  2,280  Comedy  Biuro
4                        Basketball  23 min  2005  2,505  Comedy  Biuro
5                          Hot Girl  23 min  2005  2,253  Comedy  Biuro
6                       The Dundies  21 min  2005  2,539  Comedy  Biuro
7                 Sexual Harassment  22 min  2005  2,149  Comedy  Biuro
8                   Office Olympics  22 min  2005  2,170  Comedy  Biuro
9                          The Fire  22 min  2005  2,147  Comedy  Biuro
10                        Halloween  30 min  2005  2,027  Comedy  Biuro
11                        The Fight  22 min  2005  2,026  Comedy  Biuro
12                       The Client  22 min  2005  2,070  Comedy

Create a combined dataframe of all the episodes from different tv shows

In [7]:
dfs = list(parse_and_get_df2(idn) for idn in idnlist)
if len(dfs)>1:
    print(len(dfs))
    df4=pd.concat(dfs)
else:
    df4=pd.concat(dfs)
    
df4

32


,name,len,year,votes,genre,series
0,Pilot,58 min,2008,"17,348","Crime, Drama, Thriller",Breaking Bad
1,Cat's in the Bag...,48 min,2008,"11,980","Crime, Drama, Thriller",Breaking Bad
2,...And the Bag's in the River,48 min,2008,"11,612","Crime, Drama, Thriller",Breaking Bad
3,Cancer Man,48 min,2008,"11,206","Crime, Drama, Thriller",Breaking Bad
4,Gray Matter,48 min,2008,"10,961","Crime, Drama, Thriller",Breaking Bad
5,Crazy Handful of Nothin',48 min,2008,"12,758","Crime, Drama, Thriller",Breaking Bad
6,A No-Rough-Stuff-Type Deal,48 min,2008,"11,266","Crime, Drama, Thriller",Breaking Bad
7,Seven Thirty-Seven,47 min,2009,"10,023","Crime, Drama, Thriller",Breaking Bad
8,Grilled,46 min,2009,"11,906","Crime, Drama, Thriller",Breaking Bad
9,Bit by a Dead Bee,47 min,2009,"9,718","Crime, Drama, Thriller",Breaking Bad


Split genres into different columns

In [8]:
genre_df = df4['genre'].apply(lambda x: pd.Series(x.split(',')))
genre_df.rename(columns={0:'Genre 1',1:'Genre 2',2:'Genre3'},inplace=True)

df6=pd.concat([df4,genre_df],axis=1)
df6['len']=df6['len'].str.extract('(\d+)', expand=True)
df6['votes']=df6['votes'].str.replace(',', '')
df6=df6.drop('genre',1)
df6

,name,len,year,votes,series,Genre 1,Genre 2,Genre3
0,Pilot,58,2008,17348,Breaking Bad,Crime,Drama,Thriller
1,Cat's in the Bag...,48,2008,11980,Breaking Bad,Crime,Drama,Thriller
2,...And the Bag's in the River,48,2008,11612,Breaking Bad,Crime,Drama,Thriller
3,Cancer Man,48,2008,11206,Breaking Bad,Crime,Drama,Thriller
4,Gray Matter,48,2008,10961,Breaking Bad,Crime,Drama,Thriller
5,Crazy Handful of Nothin',48,2008,12758,Breaking Bad,Crime,Drama,Thriller
6,A No-Rough-Stuff-Type Deal,48,2008,11266,Breaking Bad,Crime,Drama,Thriller
7,Seven Thirty-Seven,47,2009,10023,Breaking Bad,Crime,Drama,Thriller
8,Grilled,46,2009,11906,Breaking Bad,Crime,Drama,Thriller
9,Bit by a Dead Bee,47,2009,9718,Breaking Bad,Crime,Drama,Thriller


Following function is constructed to scrape different page and gets the following data:
    
    Series name
    Episode name
    Season number
    Episode number
    Episode rating
    
    

In [17]:
def parse_and_get_df(idn):
    page = requests.get("http://www.imdb.com/title/{}/episodes?season=1".format(idn))
    soup = BeautifulSoup(page.content, 'html.parser')
    #get how many seasons are there
    table=soup.find(id="bySeason")
    numofseas=[]
    seasnr=table.findAll('option')
    x=1
    for i in seasnr:
        numofseas.append(x)
        x+=1
    numofseas
    #now i have list of that many seasons i will loop for every of them and create dataframe and then merge it
    name=[]
    ratings=[]
    seasonlist=[]
    epnumbers=[]
    for k in numofseas:
        page = requests.get("http://www.imdb.com/title/{}/episodes?season={}".format(idn,k))
        soup = BeautifulSoup(page.content, 'html.parser')
        
        #check if season is not released yet
        table=soup.findAll('div', {"class":"ipl-rating-star--placeholder"})
        if not table:
            
            #Below give names of episodes
            table=soup.findAll('a', {"itemprop": "name" })
            for i in table:
                #print(i.get_text())
                name.append(i.get_text())
            #Below give ratings numbers
            table=soup.findAll('span', {"class": "ipl-rating-star__rating" })
            ratinglist=table[0::23]
            for i in ratinglist:
                #print(i.get_text())
                ratings.append(i.get_text())
            #Below give season number
            table=soup.findAll('h3', {"itemprop":"name"})
            season=table[1].get_text()[-1]
            for i in range(len(ratinglist)):
                seasonlist.append(season)
            #Below give episode number
            for i in range(len(ratinglist)):
                epnumbers.append(i+1)

    #Below give name of the show
    table=soup.find('a', {"class":"subnav_heading"})
    show=table.get_text()  
    showmulti=[show for i in range(len(name))]

    #create dataframe for the series
    return pd.DataFrame( OrderedDict( (('series', pd.Series(showmulti)), 
                                   ('name', pd.Series(name)), 
                                   ('season', pd.Series(seasonlist)), 
                                   ('number', pd.Series(epnumbers)), 
                                   ('rating', pd.Series(ratings))) ) )



In [18]:
%%time
df=parse_and_get_df('tt2442560')
print(df)

            series          name season  number rating
0   Peaky Blinders  Episode #1.1      1       1    8.2
1   Peaky Blinders  Episode #1.2      1       2    8.4
2   Peaky Blinders  Episode #1.3      1       3    8.5
3   Peaky Blinders  Episode #1.4      1       4    8.8
4   Peaky Blinders  Episode #1.5      1       5    9.1
5   Peaky Blinders  Episode #1.6      1       6    9.2
6   Peaky Blinders  Episode #2.1      2       1    8.7
7   Peaky Blinders  Episode #2.2      2       2    8.6
8   Peaky Blinders  Episode #2.3      2       3    8.8
9   Peaky Blinders  Episode #2.4      2       4    8.7
10  Peaky Blinders  Episode #2.5      2       5    8.9
11  Peaky Blinders  Episode #2.6      2       6    9.6
12  Peaky Blinders  Episode #3.1      3       1    8.4
13  Peaky Blinders  Episode #3.2      3       2    9.1
14  Peaky Blinders  Episode #3.3      3       3    9.0
15  Peaky Blinders  Episode #3.4      3       4    9.0
16  Peaky Blinders  Episode #3.5      3       5    8.7
17  Peaky 

Test script speed with the use of multiple threads.

In [20]:
%%time
import multiprocessing  
import random
from multiprocessing import Pool
from defs import parse_and_get_df
idnlist=['tt0903747',
'tt1632701',
'tt4158110',
'tt2575988',
'tt3322314',
'tt3322312',
'tt2357547',
'tt3322310',
'tt5675620',
'tt4230076',
'tt3032476',
'tt2442560',
'tt1837492']
p = Pool(12)  # Pool tells how many at a time
records = p.map(parse_and_get_df, idnlist)
p.terminate()
p.join()

print(records)

[          series                           name season  number rating
0   Breaking Bad                          Pilot      1       1    8.9
1   Breaking Bad            Cat's in the Bag...      1       2    8.7
2   Breaking Bad  ...And the Bag's in the River      1       3    8.7
3   Breaking Bad                     Cancer Man      1       4    8.2
4   Breaking Bad                    Gray Matter      1       5    8.3
5   Breaking Bad       Crazy Handful of Nothin'      1       6    9.2
6   Breaking Bad     A No-Rough-Stuff-Type Deal      1       7    8.8
7   Breaking Bad             Seven Thirty-Seven      2       1    8.7
8   Breaking Bad                        Grilled      2       2    9.2
9   Breaking Bad              Bit by a Dead Bee      2       3    8.4
10  Breaking Bad                           Down      2       4    8.2
11  Breaking Bad                       Breakage      2       5    8.3
12  Breaking Bad                       Peekaboo      2       6    8.8
13  Breaking Bad   

In [21]:
dfx = [parse_and_get_df(idn) for idn in idnlist]
if len(dfx)>1:
    print(len(dfx))
    df5=pd.concat(dfx)
else:
    df5=pd.concat(dfx)
df5

13


### Create final dataframe and then export to excel file

In [23]:
dffinal2=pd.merge(df5, df6, on=["series", "name"])
dffinal2[['season','number','rating','len','votes']] = dffinal2[['season','number', 'rating','len','votes']].apply(pd.to_numeric)
dffinal2

,series,name,season,number,rating,len,year,votes,Genre 1,Genre 2,Genre3
0,Breaking Bad,Pilot,1,1,8.9,58,2008,17348,Crime,Drama,Thriller
1,Breaking Bad,Cat's in the Bag...,1,2,8.7,48,2008,11980,Crime,Drama,Thriller
2,Breaking Bad,...And the Bag's in the River,1,3,8.7,48,2008,11612,Crime,Drama,Thriller
3,Breaking Bad,Cancer Man,1,4,8.2,48,2008,11206,Crime,Drama,Thriller
4,Breaking Bad,Gray Matter,1,5,8.3,48,2008,10961,Crime,Drama,Thriller
5,Breaking Bad,Crazy Handful of Nothin',1,6,9.2,48,2008,12758,Crime,Drama,Thriller
6,Breaking Bad,A No-Rough-Stuff-Type Deal,1,7,8.8,48,2008,11266,Crime,Drama,Thriller
7,Breaking Bad,Seven Thirty-Seven,2,1,8.7,47,2009,10023,Crime,Drama,Thriller
8,Breaking Bad,Grilled,2,2,9.2,46,2009,11906,Crime,Drama,Thriller
9,Breaking Bad,Bit by a Dead Bee,2,3,8.4,47,2009,9718,Crime,Drama,Thriller


In [26]:
writer = pd.ExcelWriter('datamainR.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
dffinal2.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [27]:
df=df.reset_index()
print(df.to_json(orient='index')) # export to json

{"0":{"index":0,"series":"Peaky Blinders","name":"Episode #1.1","season":"1","number":1,"rating":"8.2"},"1":{"index":1,"series":"Peaky Blinders","name":"Episode #1.2","season":"1","number":2,"rating":"8.4"},"2":{"index":2,"series":"Peaky Blinders","name":"Episode #1.3","season":"1","number":3,"rating":"8.5"},"3":{"index":3,"series":"Peaky Blinders","name":"Episode #1.4","season":"1","number":4,"rating":"8.8"},"4":{"index":4,"series":"Peaky Blinders","name":"Episode #1.5","season":"1","number":5,"rating":"9.1"},"5":{"index":5,"series":"Peaky Blinders","name":"Episode #1.6","season":"1","number":6,"rating":"9.2"},"6":{"index":6,"series":"Peaky Blinders","name":"Episode #2.1","season":"2","number":1,"rating":"8.7"},"7":{"index":7,"series":"Peaky Blinders","name":"Episode #2.2","season":"2","number":2,"rating":"8.6"},"8":{"index":8,"series":"Peaky Blinders","name":"Episode #2.3","season":"2","number":3,"rating":"8.8"},"9":{"index":9,"series":"Peaky Blinders","name":"Episode #2.4","season":